In [1]:
!python -V

Python 3.12.1


In [2]:
import pandas as pd

In [3]:
import os
import pickle

In [4]:
import seaborn as sns
import matplotlib.pyplot as plt

In [5]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error
#from sklearn.metrics import root_mean_squared_error

In [6]:
import mlflow

mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")

<Experiment: artifact_location='/workspaces/MLOps-zoomcamp/02-experiment-tracking/mlruns/1', creation_time=1727314076217, experiment_id='1', last_update_time=1727314076217, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [7]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [8]:
df_train = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-01.parquet')
df_val = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-02.parquet')

#df_train = read_dataframe('/datasets/dtc/green_tripdata_2021-01.parquet')
#df_val = read_dataframe('/datasets/dtc/green_tripdata_2021-02.parquet')

In [9]:
len(df_train), len(df_val)

(73908, 61921)

In [10]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [11]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [12]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [13]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

/home/codespace/.local/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


np.float64(7.758715209663881)

In [14]:
# Create directories if they don't exist
#os.makedirs('/models/', exist_ok=True)  # Create all subdirectories

try:
    os.makedirs('models/', exist_ok=True)
except Exception as e:
    print(f"Error creating models folder: {e}")

with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [15]:
with mlflow.start_run():

    mlflow.set_tag("developer", "adi")

    mlflow.log_param("train-data-path", "/datasets/dtc/green_tripdata_2021-01.parquet")
    mlflow.log_param("valid-data-path", "/datasets/dtc/green_tripdata_2021-02.parquet")

    alpha = 0.1
    mlflow.log_param("alpha", alpha)
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    mlflow.log_artifact(local_path="models/lin_reg.bin", artifact_path="models_pickle")

/home/codespace/.local/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [16]:
import xgboost as xgb
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [17]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [18]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            #num_boost_round=1000,  # for processor multi-core Intel i5 with 8GB RAM or higher
            num_boost_round=200,  # Reduced from 1000 for processor Intel Celeron N3350 with 4GB RAM 
            evals=[(valid, 'validation')],
            #early_stopping_rounds=50  # for processor multi-core Intel i5 with 8GB RAM or higher 
            early_stopping_rounds=20  # Reduced from 50 for processor Intel Celeron N3350 with 4GB RAM 
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)
    return {'loss': rmse, 'status': STATUS_OK}


In [19]:
search_space = {
    #'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)), # for processor multi-core Intel i5 with 8GB RAM or higher
    'max_depth': scope.int(hp.quniform('max_depth', 4, 20, 1)),  # Reduced range for processor Intel Celeron N3350 with 4GB RAM 
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    #'objective': 'reg:linear', # for processor multi-core Intel i5 with 8GB RAM or higher
    'objective': 'reg:squarederror',  # Updated objective for processor Intel Celeron N3350 with 4GB RAM 
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

[0]	validation-rmse:8.02375                                                                                                
[1]	validation-rmse:7.04875                                                                                                
[2]	validation-rmse:6.82577                                                                                                
[3]	validation-rmse:6.76391                                                                                                
[4]	validation-rmse:6.74770                                                                                                
[5]	validation-rmse:6.74098                                                                                                
[6]	validation-rmse:6.73448                                                                                                
[7]	validation-rmse:6.73059                                                                                                
[8]	vali

/home/codespace/.local/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(



[0]	validation-rmse:11.21684                                                                                               
[1]	validation-rmse:10.38496                                                                                               
[2]	validation-rmse:9.69354                                                                                                
[3]	validation-rmse:9.12237                                                                                                
[4]	validation-rmse:8.65501                                                                                                
[5]	validation-rmse:8.27333                                                                                                
[6]	validation-rmse:7.96446                                                                                                
[7]	validation-rmse:7.71508                                                                                                
[8]	vali

/home/codespace/.local/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(



[0]	validation-rmse:10.42913                                                                                               
[1]	validation-rmse:9.19538                                                                                                
[2]	validation-rmse:8.36139                                                                                                
[3]	validation-rmse:7.80896                                                                                                
[4]	validation-rmse:7.44984                                                                                                
[5]	validation-rmse:7.21908                                                                                                
[6]	validation-rmse:7.06627                                                                                                
[7]	validation-rmse:6.96544                                                                                                
[8]	vali

...

/home/codespace/.local/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(



[0]	validation-rmse:10.78999                                                                                               
[1]	validation-rmse:9.70584                                                                                                
[2]	validation-rmse:8.89471                                                                                                
[3]	validation-rmse:8.29620                                                                                                
[4]	validation-rmse:7.85750                                                                                                
[5]	validation-rmse:7.53664                                                                                                
[6]	validation-rmse:7.30452                                                                                                
[7]	validation-rmse:7.13842                                                                                                
[8]	vali

/home/codespace/.local/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(



[0]	validation-rmse:9.31414                                                                                                
[1]	validation-rmse:7.88649                                                                                                
[2]	validation-rmse:7.22718                                                                                                
[3]	validation-rmse:6.92848                                                                                                
[4]	validation-rmse:6.78504                                                                                                
[5]	validation-rmse:6.71560                                                                                                
[6]	validation-rmse:6.67518                                                                                                
[7]	validation-rmse:6.65183                                                                                                
[8]	vali

/home/codespace/.local/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(



In [20]:
mlflow.xgboost.autolog(disable=True)

In [21]:
'''
with mlflow.start_run():
    
    train = xgb.DMatrix(X_train, label=y_train)
    valid = xgb.DMatrix(X_val, label=y_val)

    best_params = {
        'learning_rate': 0.09585355369315604,
        'max_depth': 30,
        'min_child_weight': 1.060597050922164,
        'objective': 'reg:linear',
        'reg_alpha': 0.018060244040060163,
        'reg_lambda': 0.011658731377413597,
        'seed': 42
    }

    mlflow.log_params(best_params)

    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=1000,
        evals=[(valid, 'validation')],
        early_stopping_rounds=50
    )

    y_pred = booster.predict(valid)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    with open("models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)
    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

    mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")
'''

with mlflow.start_run():
    best_result['max_depth'] = int(best_result['max_depth'])  # Ensure max_depth is an integer
    mlflow.log_params(best_result)
    booster = xgb.train(
        params=best_result,
        dtrain=train,
        #num_boost_round=1000, # for processor multi-core Intel i5 with 8GB RAM or higher
        num_boost_round=200,  # Reduced from 1000 for processor Intel Celeron N3350 with 4GB RAM 
        evals=[(valid, 'validation')],
        #early_stopping_rounds=50 # for processor multi-core Intel i5 with 8GB RAM or higher
        early_stopping_rounds=20  # Reduced from 50 for processor Intel Celeron N3350 with 4GB RAM 
    )
    y_pred = booster.predict(valid)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    with open("models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)
    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

    mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")

[0]	validation-rmse:8.51474
[1]	validation-rmse:7.25042
[2]	validation-rmse:6.84703
[3]	validation-rmse:6.70877
[4]	validation-rmse:6.65305
[5]	validation-rmse:6.62711
[6]	validation-rmse:6.61426
[7]	validation-rmse:6.60734
[8]	validation-rmse:6.59650
[9]	validation-rmse:6.59525
[10]	validation-rmse:6.58831
[11]	validation-rmse:6.58327
[12]	validation-rmse:6.57861
[13]	validation-rmse:6.57422
[14]	validation-rmse:6.56965
[15]	validation-rmse:6.56766
[16]	validation-rmse:6.56394
[17]	validation-rmse:6.56007
[18]	validation-rmse:6.55550
[19]	validation-rmse:6.55099
[20]	validation-rmse:6.54778
[21]	validation-rmse:6.54319
[22]	validation-rmse:6.54138
[23]	validation-rmse:6.53829
[24]	validation-rmse:6.53394
[25]	validation-rmse:6.53112
[26]	validation-rmse:6.52572
[27]	validation-rmse:6.52303
[28]	validation-rmse:6.51998
[29]	validation-rmse:6.51825
[30]	validation-rmse:6.51551
[31]	validation-rmse:6.51274
[32]	validation-rmse:6.51148
[33]	validation-rmse:6.50872
[34]	validation-rmse:6.5

/home/codespace/.local/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/usr/local/python/3.12.1/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [02:29:51] WARNING: /workspace/src/c_api/c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  warnings.warn(smsg, UserWarning)
2024/09/26 02:29:54 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


In [ ]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import LinearSVR

mlflow.sklearn.autolog()

for model_class in (RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor, LinearSVR):

    with mlflow.start_run():

        mlflow.log_param("train-data-path", "/datasets/dtc/green_tripdata_2021-01.parquet")
        mlflow.log_param("valid-data-path", "/datasets/dtc/green_tripdata_2021-02.parquet")
        mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

        mlmodel = model_class()
        mlmodel.fit(X_train, y_train)

        y_pred = mlmodel.predict(X_val)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)
        

2024/09/26 02:29:54 WARNING mlflow.utils.autologging_utils: MLflow sklearn autologging is known to be compatible with 0.24.1 <= scikit-learn <= 1.5.1, but the installed version is 1.5.2. If you encounter errors during autologging, try upgrading / downgrading scikit-learn to a compatible version, or try upgrading MLflow.
2024/09/26 02:29:55 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'
